In [7]:
import torch
from train import train_model
from dataset.utils import *
from utils import *
import numpy as np

In [8]:
batch_size = 32
def redefine_train_parameters():
    global device, epochs, learning_rate, loss_lr, num_classes, embedding_size
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    epochs = 2
    learning_rate = 0.001
    loss_lr = 0.01
    num_classes = get_num_classes(train_dataset)
    embedding_size = 512

In [9]:
def validate(train_dataset, test_dataset, title=False):
    global r1,r2,r4,r8,r16,r32
    train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
    test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
    model = train_model(model_name='resnet50',
                    loss_func_name='proxy_anchor',
                    num_classes=num_classes,
                    dataloader=train_loader,
                    epochs=epochs,
                    learning_rate=learning_rate,
                    verbose=0)
    if title:
        print(f'Recall {title}:')
    r1,r2,r4,r8,r16,r32 = evaluate_cos(model, test_loader)
    return r1,r2,r4,r8,r16,r32

In [10]:
def cross_validate(dataset, k=10):
    global r1,r2,r4,r8,r16,r32
    k=k
    split_len = len(dataset)//k
    max_index = split_len*k
    r1,r2,r4,r8,r16,r32= [],[],[],[],[],[]
    for fold in range(k):
        print(f'''----------
fold {fold+1}:''')
        start = split_len * fold
        train_dataset = dataset[:start] + dataset[start + split_len:max_index]
        test_dataset = dataset[start:start+split_len]
        train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
        test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
        model = train_model(model_name='resnet50',
                        loss_func_name='proxy_anchor',
                        num_classes=num_classes,
                        dataloader=train_loader,
                        epochs=epochs,
                        learning_rate=learning_rate,
                        verbose=0)
        recall = evaluate_cos(model, test_loader)
        r1.append(recall[0])
        r2.append(recall[1])
        r4.append(recall[2])
        r8.append(recall[3])
        r16.append(recall[4])
        r32.append(recall[5])

In [11]:
def avg_recall(name):
    print(f'''Average Recall {name}:
R@1 : {np.mean(r1)*100:.4f}
R@2 : {np.mean(r2)*100:.4f}
R@4 : {np.mean(r4)*100:.4f}
R@8 : {np.mean(r8)*100:.4f}
R@16 : {np.mean(r16)*100:.4f}
R@32 : {np.mean(r32)*100:.4f}''')

In [12]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100)
redefine_train_parameters()

In [13]:
validate(train_dataset,test_dataset, title='PA')

Finished Training, Time: 334.3954 Best loss: 9.5777
Recall PA:


100%|██████████| 262/262 [04:02<00:00,  1.08it/s]


R@1 : 0.549
R@2 : 0.919
R@4 : 1.551
R@8 : 2.660
R@16 : 3.877
R@32 : 5.571


(0.005487295717523559,
 0.00918525587498509,
 0.015507574853870928,
 0.02660145532625552,
 0.03876893713467732,
 0.05570798043659788)

In [14]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, flip=0.2)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100)
redefine_train_parameters()

In [15]:
validate(train_dataset,test_dataset,title='PA+Flip(.2)')

Finished Training, Time: 331.9935 Best loss: 9.5116
Recall PA+Flip(.2):


100%|██████████| 262/262 [04:02<00:00,  1.08it/s]


R@1 : 0.644
R@2 : 1.050
R@4 : 1.694
R@8 : 2.756
R@16 : 4.330
R@32 : 6.239


(0.006441608016223309,
 0.010497435285697245,
 0.016939043301920552,
 0.027555767624955266,
 0.043301920553501135,
 0.062388166527496125)

In [16]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, hist='v')
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, hist='v')
redefine_train_parameters()

In [17]:
validate(train_dataset,test_dataset,title='PA+AdaptiveV')

Finished Training, Time: 415.4978 Best loss: 9.5199
Recall PA+AdaptiveV:


100%|██████████| 262/262 [05:03<00:00,  1.16s/it]


R@1 : 0.823
R@2 : 1.372
R@4 : 2.195
R@8 : 3.364
R@16 : 4.676
R@32 : 6.203


(0.00823094357628534,
 0.0137182392938089,
 0.021949182870094237,
 0.03363950852916617,
 0.04676130263628773,
 0.062030299415483714)

In [18]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median=True)
redefine_train_parameters()

In [19]:
validate(train_dataset,test_dataset,title='PA+Median')

Finished Training, Time: 434.1292 Best loss: 9.5932
Recall PA+Median:


100%|██████████| 262/262 [05:18<00:00,  1.21s/it]


R@1 : 0.656
R@2 : 1.062
R@4 : 1.622
R@8 : 2.350
R@16 : 3.638
R@32 : 5.607


(0.006560897053560778,
 0.010616724323034713,
 0.01622330907789574,
 0.023499940355481332,
 0.03638315638792795,
 0.056065847548610284)

In [20]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, hist='v', median_before=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, hist='v', median_before=True)
redefine_train_parameters()

In [21]:
validate(train_dataset,test_dataset,title='PA+Median+AdaptiveV')

Finished Training, Time: 518.2473 Best loss: 9.5689
Recall PA+Median+AdaptiveV:


100%|██████████| 262/262 [06:21<00:00,  1.46s/it]


R@1 : 1.169
R@2 : 2.052
R@4 : 3.364
R@8 : 5.153
R@16 : 7.300
R@32 : 9.615


(0.011690325659071932,
 0.020517714422044614,
 0.03363950852916617,
 0.05153286412978647,
 0.07300489085053084,
 0.09614696409399977)

In [22]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True)
redefine_train_parameters()

In [23]:
validate(train_dataset,test_dataset,title='PA+Unsharp')

/opt/conda/lib/python3.8/site-packages/skimage/filters/_unsharp_mask.py:11: FutureWarning: Automatic detection of the color channel was deprecated in v0.19, and `channel_axis=None` will be the new default in v0.22. Set `channel_axis=-1` explicitly to silence this warning.
  blurred = gaussian(image, sigma=radius, mode='reflect')


Finished Training, Time: 379.0681 Best loss: 9.5272
Recall PA+Unsharp:


100%|██████████| 262/262 [04:37<00:00,  1.06s/it]


R@1 : 0.799
R@2 : 1.300
R@4 : 1.861
R@8 : 2.899
R@16 : 4.354
R@32 : 6.096


(0.007992365501610402,
 0.013002505069784088,
 0.018609089824645115,
 0.02898723607300489,
 0.04354049862817607,
 0.0609566980794465)

In [24]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median_unsharp=True, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median=True)
redefine_train_parameters()

In [25]:
validate(train_dataset,test_dataset,title='PA+MedianUnsharp+Median')

Finished Training, Time: 492.4416 Best loss: 9.5952
Recall PA+MedianUnsharp+Median:


100%|██████████| 262/262 [06:02<00:00,  1.38s/it]


R@1 : 0.787
R@2 : 1.217
R@4 : 1.622
R@8 : 2.624
R@16 : 4.211
R@32 : 6.048


(0.007873076464272934,
 0.012167481808421806,
 0.01622330907789574,
 0.02624358821424311,
 0.04210903018012645,
 0.06047954193009662)

In [27]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, flip=.2, median_unsharp=True, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median=True)
redefine_train_parameters()

In [29]:
validate(train_dataset,test_dataset,title='PA+Flip(.2)+MedianUnsharp+Median')

Finished Training, Time: 491.8508 Best loss: 9.5984
Recall PA+Flip(.2)+MedianUnsharp+Median:


100%|██████████| 262/262 [06:02<00:00,  1.38s/it]


R@1 : 2.171
R@2 : 3.054
R@4 : 4.402
R@8 : 5.654
R@16 : 7.038
R@32 : 8.493


(0.0217106047954193,
 0.030537993558391985,
 0.04401765477752594,
 0.056543003697960156,
 0.07038053202910652,
 0.0849337945842777)

In [30]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, adjust_log=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, adjust_log=True)
redefine_train_parameters()

In [31]:
validate(train_dataset,test_dataset,title='PA+AdjustLog')

Finished Training, Time: 334.0110 Best loss: 9.5603
Recall PA+AdjustLog:


100%|██████████| 262/262 [04:04<00:00,  1.07it/s]


R@1 : 0.919
R@2 : 1.491
R@4 : 2.302
R@8 : 3.233
R@16 : 4.414
R@32 : 6.525


(0.00918525587498509,
 0.014911129667183587,
 0.023022784206131457,
 0.03232732911845401,
 0.04413694381486341,
 0.06525110342359537)

In [32]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True, adjust_log=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True, adjust_log=True)
redefine_train_parameters()

In [33]:
validate(train_dataset,test_dataset,title='PA+Unsharp+AdjustLog')

Finished Training, Time: 381.9993 Best loss: 9.5122
Recall PA+Unsharp+AdjustLog:


100%|██████████| 262/262 [04:40<00:00,  1.07s/it]


R@1 : 1.062
R@2 : 1.515
R@4 : 2.243
R@8 : 3.042
R@16 : 4.330
R@32 : 5.988


(0.010616724323034713,
 0.015149707741858523,
 0.022426339019444113,
 0.030418704521054517,
 0.043301920553501135,
 0.05988309674340928)

In [34]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, adjust_log=True, unsharp_after=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, adjust_log=True, unsharp_after=True)
redefine_train_parameters()

In [35]:
validate(train_dataset,test_dataset,title='PA+AdjustLog+Unsharp')

Finished Training, Time: 381.3271 Best loss: 9.4625
Recall PA+AdjustLog+Unsharp:


100%|██████████| 262/262 [04:39<00:00,  1.07s/it]


R@1 : 0.871
R@2 : 1.324
R@4 : 1.956
R@8 : 2.768
R@16 : 4.115
R@32 : 5.404


(0.008708099725635214,
 0.013241083144459024,
 0.019563402123344866,
 0.027675056662292735,
 0.041154717881426696,
 0.054037933913873314)